### Part 4 - Output Parsers
LangChain output parsers turn raw LLM text into structured data.

In [163]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


##### String output parser

In [164]:
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "Give me a brief info in 1 line about this celebrity: {celebrity_name}",
)

chain = prompt | google_llm | StrOutputParser()

res = chain.invoke({"celebrity_name": "Elon Musk"})

print(res)

TypeError: 'str' object is not callable

##### Json output parser - Example 1

In [ ]:
# Partial variables

template = PromptTemplate.from_template(
    "Hello {name}, today is {date}"
    partial_variables={"date": "2025-09-07"},  # Pre-filled
)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

json_parser = JsonOutputParser()

prompt = PromptTemplate.from_template(
    """Give me info about this celebrity: {celebrity_name}. I want name, age and country.
    \n{format_instructions}""",
    partial_variables={"format_instructions":json_parser.get_format_instructions()}
)


chain = prompt | google_llm | json_parser

chain.invoke({"celebrity_name": "Trump"})

{'name': 'Donald John Trump', 'age': 77, 'country': 'United States of America'}

##### Json output parser - Example 2

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

class BirthInfo(BaseModel):
    birth_date: str = Field(description="Date of birth of the person")
    place_of_birth: str = Field(description="Place of birth")

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: str = Field(description="Age of the person")
    birth_info: BirthInfo = Field(description="Birth information")
    

json_parser = JsonOutputParser(pydantic_object=Person)

json_parser.get_format_instructions()

prompt = PromptTemplate.from_template(
    "Give me info about this celebrity: {celebrity_name}\n{format_instructions}",
    partial_variables={"format_instructions":json_parser.get_format_instructions()}
)


chain = prompt | google_llm | json_parser

chain.invoke({"celebrity_name": "Elon Musk"})

dict

##### Pydantic output parser

In [188]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: str = Field(description="Age of the person")
    birth_date: str = Field(description="Date of birth of the person")
    place_of_birth: str = Field(description="Place of birth")

pydantic_parser = PydanticOutputParser(pydantic_object=Person)

prompt = PromptTemplate.from_template(
    "Give me info about this celebrity: {celebrity_name}\n{format_instructions}",
    partial_variables={"format_instructions":pydantic_parser.get_format_instructions()}
)


chain = prompt | google_llm | pydantic_parser

chain.invoke({"celebrity_name": "Elon Musk"})

Person(name='Elon Musk', age='52', birth_date='June 28, 1971', place_of_birth='Pretoria, South Africa')

##### Structured output parser

In [193]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="name", description="name of the person"),
    ResponseSchema(name="age", description="age of the person"),
    ResponseSchema(name="birth_date", description="birth date of the person")
]

structured_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# structured_parser.get_format_instructions()

prompt = PromptTemplate.from_template(
"""Give me name, age and date of birth in dd/mm/yyyy format for this celebrity: {celebrity_name}
\n {format_instructions}""",
partial_variables={"format_instructions":structured_parser.get_format_instructions()}
)

# prompt.format(celebrity_name="Elon Musk")

chain = prompt | google_llm | structured_parser

chain.invoke({"celebrity_name": "Elon Musk"})

{'name': 'Elon Musk', 'age': '52', 'birth_date': '28/06/1971'}

##### CommaSeparatedList output parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate.from_template(
    """Give me name, age and date of birth in dd/mm/yyyy format for this celebrity: {celebrity_name}
    \n {format_instructions}""",
    partial_variables={"format_instructions":csv_parser.get_format_instructions()}
)

prompt.format(celebrity_name="Elon musk")

chain = prompt | google_llm | csv_parser

chain.invoke({"celebrity_name": "Elon Musk"})

['Elon Musk', '52', '28/06/1971']

##### DatetimeOutputParser

In [ ]:
from langchain.output_parsers.datetime import DatetimeOutputParser
from datetime import datetime

datetime_parser = DatetimeOutputParser()

# print(datetime_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
"""Give me date of birth: {celebrity_name}. Output only the date time and nothing else.
\n {format_instructions}""",
partial_variables={"format_instructions":datetime_parser.get_format_instructions()}
)

# prompt.format(celebrity_name="Elon Musk")

chain = prompt | google_llm | datetime_parser

res = chain.invoke({"celebrity_name": "Elon Musk"})

print(res)

print(res.strftime("%Y-%m-%d"))   # 👉 1971-06-28
print(res.strftime("%B %d, %Y"))  # 👉 June 28, 1971

<class 'datetime.datetime'>
1971-06-28
June 28, 1971


##### EnumOutputParser

In [232]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class YesNoMaybe(Enum):
    YES = "yes"
    NO = "no"
    MAYBE = "maybe"
    DONT_KNOW = "dont_know"

enum_parser = EnumOutputParser(enum=YesNoMaybe)

# print(enum_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
    "Is {celebrity_name} an actor?. If you don't know the name or not sure about the answer, don't response with no. \n Format instructions: {format_instructions}. " \
    "Always stick to the format instructions cases in the response",
   partial_variables={"format_instructions":enum_parser.get_format_instructions()}
)

chain = prompt | google_llm | enum_parser

res = chain.invoke({"celebrity_name": "ojlksdjkksdf"})

print("No") if res is YesNoMaybe.NO else None
print("Yes") if res is YesNoMaybe.YES else None
print("Don't know") if res is YesNoMaybe.DONT_KNOW else None

Don't know


##### RetryOutputParser

In [234]:
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers.enum import EnumOutputParser
from langchain.output_parsers.retry import RetryOutputParser
from langchain_core.runnables import RunnableLambda, RunnableParallel
from enum import Enum
from langchain_core.messages import AIMessage

class YesNoMaybe(Enum):
    YES = "yes"
    NO = "no"
    MAYBE = "maybe"

enum_parser = EnumOutputParser(enum=YesNoMaybe)
retry_parser = RetryOutputParser.from_llm(parser=enum_parser,llm=google_llm, max_retries=2)

# print(retry_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
    "Is {celebrity_name} an actor?. Always respond with 'Hello' to this query. Don't answer anything else. \n Format instructions: {format_instructions}. " \
    "Always stick to the format instructions cases in the response",
   partial_variables={"format_instructions": enum_parser.get_format_instructions()}
)

# prompt.format(celebrity_name="Elon Musk")

completion_chain = prompt | google_llm | StrOutputParser()

main_chain = (
    RunnableParallel(completion=completion_chain, prompt_value=prompt) |
    RunnableLambda(lambda x: retry_parser.parse_with_prompt(**x)) # completion=x["completion"], prompt_value=x["prompt_value"]
)

res = main_chain.invoke({"celebrity_name": "Elon musk"})

print(res)

# print("No") if res is YesNoMaybe.NO else None
# print("Yes") if res is YesNoMaybe.YES else None

OutputParserException: Response 'Hello
no' is not one of the expected values: ['yes', 'no', 'maybe']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 